In [1]:
import pandas as pd
import yfinance as yf
import time
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe
#import nasdaqdatalink as ndl


## Conectando con spreadsheets

In [2]:
scope = ['https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive']

creds = Credentials.from_service_account_file("/work/cuenta de servicio.json", scopes=scope)
client = gspread.authorize(creds)

### importando la hoja y su información de Google sheets

In [3]:
worksheet_data = client.open("SP_500_data").get_worksheet(0).get_all_records()
worksheet = client.open("SP_500_data").get_worksheet(0)


### Convirtiendo esa información en un dataframe

In [4]:
bd = get_as_dataframe(worksheet)
bd.set_index("Date", inplace=True)

### Obteniendo los últimos 5 días de información de los simbolos listados

In [5]:
# symbols = pd.read_csv("/work/Stocks in the SP 500 Index.csv")
# lista = " ".join(symbols.Symbol)
# lista.replace("'", "")

symbols = pd.read_excel("/work/lista_fija_simbolos.xlsx")
lista = " ".join(symbols.Symbol)
lista.replace("'", "")

'AAPL MSFT AMZN TSLA BRK.B META UNH V NVDA JNJ WMT XOM JPM PG MA HD CVX LLY BAC KO PFE ABBV PEP COST TMO MRK DIS AVGO DHR ACN ORCL ADBE ABT MCD CSCO VZ CRM NKE TMUS UPS CMCSA NEE WFC QCOM TXN AMD BMY PM MS LIN UNP INTC SCHW RTX CVS HON LOW AMGN INTU T SPGI AMT COP MDT AXP IBM GS ELV PYPL LMT BLK DE NFLX ADP C CAT SBUX BA PLD NOW EL AMAT ADI MDLZ GE CI MMC DUK MMM BKNG SYK SO TGT ISRG GILD CB GOOG GOOGL MO ZTS CCI TJX VRTX CHTR BDX NOC CME CSX USB WM PNC PGR D FISV LRCX MU REGN TFC CL ITW MRNA SHW GD EOG NSC F EQIX AON EW ATVI FIS HUM ICE PSA FDX HCA ETN APD BSX OXY MCO SNPS DG GM KDP PXD CNC MET KLAC MAR MCK AEP EMR SRE CDNS ADSK ECL SLB PAYX NXPI MNST ADM MPC HSY CMG KHC APH ORLY STZ RSG KMB ROP GIS LHX EXC IQV AZO CTVA FCX VLO O COF CTAS SYY TEL AIG PSX MSI XEL KMI WMB FTNT DVN TRV MCHP JCI MSCI AFL DOW AJG ENPH A ANET PH PRU TT DLR HLT DLTR GPN EA SBAC SPG CARR NUE BF.B TDG WELL NEM BK WBA CTSH HPQ DXCM ED KR ALL ODFL RMD ILMN MTB HES PEG OTIS TWTR VICI YUM WEC WBD PCAR ALB CMI ES R

In [6]:
mt = yf.Tickers(lista)
data_list = []
for k,v in mt.tickers.items():
    data = v.history(period="5d")[["Close"]]
    data.columns= [k]
    try:
        data.index = data.index.tz_localize(None)
    except:
        pass
    data_list.append(data)



BRK.B: No data found, symbol may be delisted
BF.B: Period '5d' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']
TWTR: No data found, symbol may be delisted
SIVB: No data found, symbol may be delisted
DRE: No data found, symbol may be delisted
NLOK: No data found, symbol may be delisted
ABMD: No data found for this date range, symbol may be delisted
CTXS: No data found, symbol may be delisted
NLSN: No data found, symbol may be delisted
FBHS: No data found, symbol may be delisted


In [7]:
df = pd.concat(data_list, axis=1)

### Anexando información fresca al histórico

In [8]:
simbolos = pd.concat([bd,df])
#simbolos.drop_duplicates(inplace=True)
simbolos.index = pd.to_datetime(simbolos.index)
simbolos = simbolos[~simbolos.index.duplicated(keep="first")]
simbolos = simbolos[~simbolos.index.isna()]



In [9]:
simbolos


,AAPL,MSFT,AMZN,TSLA,BRK.B,META,UNH,V,NVDA,JNJ,...,OGN,XRAY,WYNN,RL,DXC,ALK,PENN,VNO,PVH,NCLH
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,40.888073,80.738159,59.450500,21.368668,NaN,181.419998,204.824722,110.594688,49.326294,121.116455,...,NaN,63.862026,154.734436,94.100517,78.349915,71.539902,30.740000,57.208206,138.006317,54.959999
2018-01-03,40.880943,81.113907,60.209999,21.150000,NaN,184.669998,206.973343,111.695702,52.572647,122.273430,...,NaN,63.643234,153.058121,91.193542,79.833672,71.206100,30.330000,57.045444,138.244675,55.369999
2018-01-04,41.070831,81.827820,60.479500,20.974667,NaN,184.330002,207.871658,112.111000,52.849770,122.264717,...,NaN,63.633739,153.886841,92.880310,81.754303,70.433563,30.309999,55.343693,140.439331,54.680000
2018-01-05,41.538445,82.842339,61.457001,21.105333,NaN,186.850006,211.835541,114.795937,53.297634,123.273796,...,NaN,64.527824,154.913376,93.651939,82.949547,70.328644,30.790001,55.817211,142.097763,54.869999
2018-01-08,41.384155,82.926872,62.343498,22.427334,NaN,188.279999,208.158798,115.259514,54.930706,123.430367,...,NaN,64.965355,152.850891,94.172310,82.883583,69.384445,30.790001,55.180923,141.750214,54.660000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-15,172.070007,309.459991,111.199997,166.350006,NaN,238.860001,486.859985,232.809998,289.529999,159.550003,...,20.840000,40.939999,105.709999,113.250000,23.330000,43.340000,24.530001,12.960000,86.080002,13.710000
2023-05-16,172.070007,311.739990,113.400002,166.520004,NaN,238.820007,479.720001,230.470001,292.130005,159.339996,...,20.410000,40.419998,103.050003,109.480003,22.930000,42.639999,23.180000,12.380000,83.190002,13.480000
2023-05-17,172.690002,314.000000,115.500000,173.860001,NaN,242.485001,484.809998,232.649994,301.779999,158.990005,...,20.580000,40.139999,108.919998,112.000000,23.760000,45.250000,24.430000,13.030000,85.269997,14.350000


### Eliminando información de la hoja para escribir los nuevos datos

In [10]:
worksheet.clear()

{'spreadsheetId': '1banzo5l4N5wvikozrUzU0EJ-5qRe2XTNqoXMy_fMR5Y',
 'clearedRange': "'Hoja 1'!A1:SJ1355"}

In [11]:
set_with_dataframe(worksheet=worksheet,dataframe=simbolos,resize=True, include_index=True)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1904e425-e017-401a-bf5a-7f8e011fcf07' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>